# Задание 1
Из наборов изображений, которые полученные в лабораторной работе № 11, выбрать по 135 изображений каждой цифры без шума: a)100/15/20 для обучающей/валидационной/тестовой выборок). В каждой выборке должно содержаться равное количество изображений каждой цифры в вертикальном и горизонтальном положении соответственно.

In [ ]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import os
import sklearn
import shutil
import tensorflow
from tensorflow import keras 
#from keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.models import Model
from sklearn import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import sys
print(sys.executable) 
import tensorflow

/usr/bin/python3


In [ ]:
def print_img(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# Количество изображений для обучения
nb_train_samples = 700
# Количество изображений для проверки
nb_validation_samples = 100
# Количество изображений для тестирования
nb_test_samples = 20
n = nb_train_samples + nb_validation_samples + nb_test_samples 

In [ ]:
config = {
    'thickness': 1,
    'data_dir': r'./data',
    'val_dir': r'./data/val',
    'train_dir': r'./data/train',
    'test_dir1': r'./data/test1',
    'test_dir2': r'./data/test2',
    'test_dir3': r'./data/test3',
    'test_dir4': r'./data/test4',
    'test_dir5': r'./data/test5',
    'nb_images': n,
}

In [ ]:
def draw0(dc, thickness):
    cv2.ellipse(dc, ((10, 30), (19, 59), 0), (0, 0, 0), thickness)


def draw1(dc, thickness):
    cv2.line(dc, (0, 20), (19, 0), (0, 0, 0), thickness)
    cv2.line(dc, (19, 0), (19, 59), (0, 0, 0), thickness)


def draw3(dc, thickness):
    cv2.ellipse(dc, (10, 22), (9, 9), 210, 0, 255, (0, 0, 0), thickness)
    cv2.ellipse(dc, (10, 40), (9, 9), 255, 0, 255, (0, 0, 0), thickness)


def draw8(dc, thickness):
    cv2.ellipse(dc, (10, 22), (9, 9), 0, 0, 360, (0, 0, 0), thickness)
    cv2.ellipse(dc, (10, 40), (9, 9), 0, 0, 360, (0, 0, 0), thickness)


In [ ]:
def create_digits(thickness):
    _dc = {
        0: np.ones((60, 20, 3), np.uint8) * 255,
        1: np.ones((60, 20, 3), np.uint8) * 255,
        3: np.ones((60, 20, 3), np.uint8) * 255,
        8: np.ones((60, 20, 3), np.uint8) * 255,
    }
    draw0(_dc[0], thickness)
    draw1(_dc[1], thickness)
    draw3(_dc[3], thickness)
    draw8(_dc[8], thickness)
    return _dc


In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "0"))
    os.makedirs(os.path.join(dir_name, "1"))
    os.makedirs(os.path.join(dir_name, "3"))
    os.makedirs(os.path.join(dir_name, "8"))


In [ ]:
def rand_point():
    return np.random.randint(0, 79), np.random.randint(0, 39)


def generate_dataset(d, _dc, _config):
    for i in range(_config['nb_images']):
        _canvas = np.ones((100, 100, 3), np.uint8) * 255
        _x, _y = rand_point()
        _canvas[_y:_y + _dc[d].shape[0], _x:_x + _dc[d].shape[1]] = _dc[d]
        if (i >= nb_train_samples//2  and i < nb_train_samples) or (i >= nb_validation_samples//2 + nb_train_samples and i <nb_validation_samples + nb_train_samples)  or  i >= nb_train_samples +  nb_validation_samples  + nb_test_samples //2:
            _canvas = np.rot90(_canvas)
     #   if i == 501:
      #      print_img(_canvas)
        cv2.imwrite(os.path.join(config['data_dir'], str(d) + '_' + str(i) + ".jpg"), _canvas)

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "0_" + str(i) + ".jpg"),os.path.join(dest_dir, "0"))
        shutil.copy2(os.path.join(source_dir, "1_" + str(i) + ".jpg"),os.path.join(dest_dir, "1"))
        shutil.copy2(os.path.join(source_dir, "3_" + str(i) + ".jpg"),os.path.join(dest_dir, "3"))
        shutil.copy2(os.path.join(source_dir, "8_" + str(i) + ".jpg"),os.path.join(dest_dir, "8"))


In [ ]:
create_directory(config["train_dir"])
create_directory(config["val_dir"])
create_directory(config["test_dir1"])
create_directory(config["test_dir2"])
create_directory(config["test_dir3"])
create_directory(config["test_dir4"])
create_directory(config["test_dir5"])


In [ ]:
dc = create_digits(config['thickness'])
generate_dataset(0, dc, config)
generate_dataset(1, dc, config)
generate_dataset(3, dc, config)
generate_dataset(8, dc, config)

In [ ]:
start_val_data_idx = nb_train_samples
start_test_data_idx =nb_train_samples + nb_validation_samples
copy_images(0, start_val_data_idx, config['data_dir'], config['train_dir'])
copy_images(start_val_data_idx, start_test_data_idx, config['data_dir'],  config['val_dir'])
copy_images(start_test_data_idx, n, config['data_dir'],  config['test_dir1'])

def pxl_gen(count, img):
    for i in range(count):
        x = np.random.randint(0, 100)
        y = np.random.randint(0, 100)
        img[x][y] = (0, 0, 0)

# Задание 2
Используя keras или torch и по очереди каждую из сверточных нейронных сетей https://keras.io/api/applications/ : Xception, ResNet152V2, InceptionResNetV2, DenseNet201, NASNetLarge дообучить сеть на этих картинках и вывести результаты классификации на тестовом множестве.

In [ ]:
data_dir = r'./data/train'
# Каталог с данными для обучения
train_dir = r'./data/train'
# Каталог с данными для проверки
val_dir = r'./data/val'
# Каталог с данными для тестирования
test_dir = r'./data/test1'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = n
# Размеры изображения
img_width, img_height = 100, 100


In [ ]:
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 15



In [ ]:
from tensorflow.keras.applications.xception import Xception

# Модель
#res_net50 = Xception(weights='imagenet',
 #                    include_top=False,
 #                    input_shape=input_shape,
 #                    classes=4,
 #                    pooling='avg')
#res_net50 = tensorflow.keras.applications.ResNet152V2(weights='imagenet',
#                   include_top=False,
#                  input_shape=input_shape,
#                classes=4,
#               pooling='avg')
#res_net50 = tensorflow.keras.applications.InceptionResNetV2(weights='imagenet',
#                   include_top=False,
#                  input_shape=input_shape,
#                classes=4,
#               pooling='avg')
#res_net50 = tensorflow.keras.applications.DenseNet201(weights='imagenet',
#                   include_top=False,
#                  input_shape=input_shape,
#                classes=4,
#               pooling='avg')
res_net50 = tensorflow.keras.applications.NASNetLarge(weights=None,include_top=False,input_shape=input_shape, classes=4,pooling='avg')
res_net50.trainable = False

In [ ]:
#from tensorflow.keras import Sequentia
model = keras.Sequential()
model.add(res_net50)
model.add(Dense(4))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 4032)              84916818  
                                                                 
 dense_16 (Dense)            (None, 4)                 16132     
                                                                 
 activation_1868 (Activation  (None, 4)                0         
 )                                                               
                                                                 
Total params: 84,932,950
Trainable params: 16,132
Non-trainable params: 84,916,818
_________________________________________________________________
None


In [ ]:
# Компилируем нейронную сеть
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# Генератор изображений создается на основе класса ImageDataGenerator.
# Генератор делит значения всех пикселов изображения на 255.
datagen = ImageDataGenerator(rescale=1. / 255)

# Генератор данных для обучения на основе изображений из каталога
train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(img_width, img_height),
                                              batch_size=batch_size,
                                              class_mode='categorical')

# Генератор данных для проверки на основе изображений из каталога
val_generator = datagen.flow_from_directory(val_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode='categorical')

# Генератор данных для тестирования на основе изображений из каталога
test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(img_width, img_height),
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             shuffle=False)

Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
# Обучаем модель с использованием генераторов
# train_generator - генератор данных для обучения
# validation_data - генератор данных для проверки
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=nb_validation_samples // batch_size)

# Оцениваем качество работы сети с помощью генератора

scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1] * 100))


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


46/46 [==============================] - 128s 2s/step - loss: 3.0038 - accuracy: 0.2362 - val_loss: 1.5221 - val_accuracy: 0.2333
Epoch 2/10
46/46 [==============================] - 97s 2s/step - loss: 1.8211 - accuracy: 0.2681 - val_loss: 2.2040 - val_accuracy: 0.2667
Epoch 3/10
46/46 [==============================] - 97s 2s/step - loss: 1.8240 - accuracy: 0.2681 - val_loss: 1.7099 - val_accuracy: 0.2333
Epoch 4/10
46/46 [==============================] - 97s 2s/step - loss: 1.6780 - accuracy: 0.2899 - val_loss: 1.6748 - val_accuracy: 0.2222
Epoch 5/10
46/46 [==============================] - 97s 2s/step - loss: 1.5440 - accuracy: 0.3797 - val_loss: 1.6189 - val_accuracy: 0.5111
Epoch 6/10
46/46 [==============================] - 100s 2s/step - loss: 1.3749 - accuracy: 0.4174 - val_loss: 1.0961 - val_accuracy: 0.5444
Epoch 7/10
46/46 [==============================] - 99s 2s/step - loss: 1.6866 - accuracy: 0.3188 - val_loss: 2.2644 - val_accuracy: 0.4556
Epoch 8/10
46/46 [===========

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  if sys.path[0] == '':


Аккуратность на тестовых данных: 0.00%


In [ ]:
mpg = model.predict_generator(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i, :]) for i in range(mpg.shape[0])])
cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       1.00      0.80      0.89        20
           3       0.00      0.00      0.00        20
           8       0.31      1.00      0.48        20

    accuracy                           0.45        80
   macro avg       0.33      0.45      0.34        80
weighted avg       0.33      0.45      0.34        80



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sklearn.metrics.confusion_matrix(y_t,y_p)


array([[ 0,  0,  0, 20],
       [ 0, 16,  0,  4],
       [ 0,  0,  0, 20],
       [ 0,  0,  0, 20]])